In [1]:
import os
os.chdir("../") # to the prev. dir
%pwd

'c:\\Users\\15600\\Desktop\\PY\\kidney-disease-classification-project'

In [3]:
### DO NOT SHARE TO OTHERS ###
### DO NOT SHARE TO OTHERS ###
### DO NOT SHARE TO OTHERS ###

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/yhr0864/kidney-disease-classification-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="yhr0864"
os.environ["MLFLOW_TRACKING_PASSWORD"]="d9d7c85432b94cdcba7ac783932b5090561e756d"

In [ ]:
MLFLOW_TRACKING_URI=https://dagshub.com/yhr0864/kidney-disease-classification-project.mlflow \
MLFLOW_TRACKING_USERNAME=yhr0864 \
MLFLOW_TRACKING_PASSWORD=d9d7c85432b94cdcba7ac783932b5090561e756d \
python script.py

In [4]:
import torch

model = torch.load("artifacts/model_training/model.pt")

## Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TestingConfig:
    model_path: Path
    testing_data: Path
    all_params: dict
    mlflow_uri: str

## Configuration

In [6]:
from src.KDClassifier.constants import *
from src.KDClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    '''
    read the config.yaml
    return base_model_config
    '''
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                       params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_testing_config(self) -> TestingConfig:
        testing_data = os.path.join(self.config.data_ingestion.unzip_dir, "test")
        test_config = TestingConfig(
            model_path=Path(self.config.model_training.trained_model_path),
            testing_data=Path(testing_data),
            all_params=self.params,
            mlflow_uri=str(self.config.model_testing.mlflow_uri)

        )
        return test_config

## Component

In [11]:
import mlflow
from urllib.parse import urlparse
import tqdm
from torch.utils.data import DataLoader
from transformers import ViTFeatureExtractor

from src.KDClassifier.utils.dataloader import KidneyDataset
from src.KDClassifier import logger
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_recall_fscore_support

class Testing:
    def __init__(self, config: TestingConfig):
        self.config = config
    
    def get_testing_model(self):
        self.model = torch.load(self.config.model_path)

    def test_generator(self):
        feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

        test_dataset = KidneyDataset(self.config.testing_data, feature_extractor)   
        self.test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    def get_testing(self):
        self.model.eval()

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)

        all_preds = []
        all_labels = []
        with torch.no_grad():
            for (inputs, labels) in tqdm.tqdm(self.test_loader,
                                              total=len(self.test_loader)):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = self.model(inputs)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        self.accuracy = accuracy_score(all_labels, all_preds)
        self.precision, self.recall, self.f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        logger.info(f"Accuracy: {self.accuracy}\nPrecision: {self.precision}\nRecall: {self.recall}\nF1 Score: {self.f1}")
        logger.info(classification_report(all_labels, all_preds, target_names=['Normal', 'Tumor']))

        # Confusion Matrix
        conf_matrix = confusion_matrix(all_labels, all_preds)
        logger.info(f"conf_matrix:\n {conf_matrix}")
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"Accuracy": self.accuracy,
                                "Precision": self.precision,
                                "Recall": self.recall,
                                "F1": self.f1

            })
            # model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.pytorch.log_model(self.model, "model", registered_model_name="ViT")
            else:
                mlflow.pytorch.log_model(self.model, "model")
            

    


## Pipeline

In [14]:
try:
    config = ConfigurationManager()
    test_config = config.get_testing_config()
    testing = Testing(test_config)
    testing.get_testing_model()
    testing.test_generator()
    testing.get_testing()
    testing.log_into_mlflow()

except Exception as e:
    raise e

[2024-02-12 16:05:06,613: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-12 16:05:06,615: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-12 16:05:06,616: INFO: common: created directory at: artifacts]


d:\anaconda3\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
100%|██████████| 200/200 [00:12<00:00, 16.10it/s]

[2024-02-12 16:05:19,769: INFO: 2221389218: Accuracy: 0.655
Precision: 0.7958579881656804
Recall: 0.655
F1 Score: 0.6083884332699566]
[2024-02-12 16:05:19,769: INFO: 2221389218:               precision    recall  f1-score   support

      Normal       1.00      0.31      0.47       100
       Tumor       0.59      1.00      0.74       100

    accuracy                           0.66       200
   macro avg       0.80      0.66      0.61       200
weighted avg       0.80      0.66      0.61       200
]
[2024-02-12 16:05:19,769: INFO: 2221389218: conf_matrix:
 [[ 31  69]
 [  0 100]]]



Successfully registered model 'ViT'.
2024/02/12 16:07:12 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ViT, version 1
Created version '1' of model 'ViT'.
